# A Macroscopically Estimating Model with Endogeneity Considered for Lane-mean Speeds and Speed Deviations (5lanes)

## 1 Structuring Data

### 1.1 Import Data

In [1]:
import numpy as np
import pandas as pd
import xlrd
from linearmodels import IV3SLS
import datetime
import math
import os
from dateutil.relativedelta import * # for incement of month

In [2]:
# filelist = []
# file_dir = 'G:\\Modeling Endogeneity\\Data'

# for root, dirs, files in os.walk(file_dir):
#     filelist = files

# file_200m_2lanes = [file for file in filelist if '200m_2lanes' in file]

In [3]:
data_up_fs = pd.DataFrame()
data_do_fs = pd.DataFrame()
data_up_o = pd.DataFrame()
data_up_tftp = pd.DataFrame()
data_up_vmtvht = pd.DataFrame()

# 12 for the number of month
for num in range(11): # 400m_5lanes lacks of the data of Dec.
    monnum = num + 1
    # flow,speed
    file_up_fs = 'G:\\Modeling Endogeneity\\Data\\400m_5lanes_up_fs_' + str(monnum) + '.xlsx'
    file_do_fs = 'G:\\Modeling Endogeneity\\Data\\400m_5lanes_do_fs_' + str(monnum) + '.xlsx'
    # occupancy
    file_up_o = 'G:\\Modeling Endogeneity\\Data\\400m_5lanes_up_o_' + str(monnum) + '.xlsx'
    file_do_o = 'G:\\Modeling Endogeneity\\Data\\400m_5lanes_do_o_' + str(monnum) + '.xlsx'
    # truck flow, truck percentage
    file_up_tftp = 'G:\\Modeling Endogeneity\\Data\\400m_5lanes_up_tftp_' + str(monnum) + '.xlsx'
    # file_do_tftp = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_do_tftp_1.xlsx'
    # truck vmt, trauck vht
    file_up_vmtvht = 'G:\\Modeling Endogeneity\\Data\\400m_5lanes_up_vmtvht_' + str(monnum) + '.xlsx'
    # file_do_vmtvht = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_do_vmtvht_1.xlsx'

    data_up_fs_t = pd.read_excel(file_up_fs)
    data_do_fs_t = pd.read_excel(file_do_fs)
    data_up_o_t = pd.read_excel(file_up_o)
    # data_do_o = pd.read_excel(file_do_o)
    data_up_tftp_t = pd.read_excel(file_up_tftp)
    # data_do_tftp = pd.read_excel(file_do_tftp)
    data_up_vmtvht_t = pd.read_excel(file_up_vmtvht)
    # data_do_vmtvht = pd.read_excel(file_do_vmtvht)
    data_up_fs = data_up_fs.append(data_up_fs_t, sort=True)
    data_do_fs = data_do_fs.append(data_do_fs_t, sort=True)
    data_up_o = data_up_o.append(data_up_o_t, sort=True)
    data_up_tftp = data_up_tftp.append(data_up_tftp_t, sort=True)
    data_up_vmtvht = data_up_vmtvht.append(data_up_vmtvht_t, sort=True)

In [4]:
data_up_vmtvht.head() # this command lane is used to observe the order of variables in the dataframe for changing the keyword easier.

,# Lane Points,% Observed,5 Minutes,Lane 1 Truck VHT (Veh-Hours),Lane 1 Truck VMT (Veh-Miles),Lane 2 Truck VHT (Veh-Hours),Lane 2 Truck VMT (Veh-Miles),Lane 3 Truck VHT (Veh-Hours),Lane 3 Truck VMT (Veh-Miles),Lane 4 Truck VHT (Veh-Hours),Lane 4 Truck VMT (Veh-Miles),Lane 5 Truck VHT (Veh-Hours),Lane 5 Truck VMT (Veh-Miles),Truck VHT (Veh-Hours),Truck VMT (Veh-Miles)
0,4,100,1/08/2017 0:00,0,0,0,0,0.0,1.1,0.0,2.2,0.0,0.0,0.1,3.3
1,4,100,1/08/2017 0:05,0,0,0,0,0.0,1.8,0.0,1.8,0.0,0.0,0.1,3.7
2,4,100,1/08/2017 0:10,0,0,0,0,0.0,1.1,0.0,1.5,0.0,0.0,0.0,2.6
3,4,100,1/08/2017 0:15,0,0,0,0,0.0,0.7,0.0,1.1,0.0,0.0,0.0,1.8
4,4,100,1/08/2017 0:20,0,0,0,0,0.0,0.7,0.0,1.5,0.0,0.0,0.0,2.2


In [5]:
data_do_fs.describe()

,# Lane Points,% Observed,Flow (Veh/5 Minutes),Lane 1 Flow (Veh/5 Minutes),Lane 1 Speed (mph),Lane 2 Flow (Veh/5 Minutes),Lane 2 Speed (mph),Lane 3 Flow (Veh/5 Minutes),Lane 3 Speed (mph),Lane 4 Flow (Veh/5 Minutes),Lane 4 Speed (mph),Lane 5 Flow (Veh/5 Minutes),Lane 5 Speed (mph),Speed (mph)
count,22162.000000,22162.000000,22162.000000,22162.000000,22162.000000,22162.000000,22162.000000,22162.000000,22162.000000,22162.000000,22162.000000,22162.000000,22162.000000,22162.000000
mean,4.908808,78.842388,293.488990,62.894414,70.092803,78.435475,68.881712,61.704765,67.141914,49.230169,59.430376,41.224167,56.080985,64.740294
std,0.289452,7.583824,194.912114,48.251481,12.179086,50.821609,7.187331,40.842648,7.209813,25.884132,4.211891,36.963646,18.992804,6.408918
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,80.000000,142.000000,20.000000,66.800000,42.000000,65.500000,34.000000,64.500000,30.000000,57.600000,3.000000,59.100000,62.400000
50%,5.000000,80.000000,283.000000,57.000000,75.300000,82.000000,71.700000,62.000000,69.300000,48.000000,60.800000,35.000000,63.600000,66.100000
75%,5.000000,80.000000,431.000000,103.000000,76.500000,112.000000,73.800000,87.000000,71.900000,65.000000,62.400000,65.000000,64.500000,69.400000
max,5.000000,80.000000,762.000000,211.000000,83.400000,207.000000,78.000000,177.000000,81.800000,122.000000,66.900000,158.000000,72.600000,75.000000


In [6]:
# Caution: use dataframe.head() to observe the name and the position of every column
data_up = data_up_fs
data_up = data_up.iloc[:,2:15]
data_up.columns = ['t','f','f1','s1','f2','s2','f3','s3','f4','s4','f5','s5','s']
data_up[['o1','o2','o3','o4','o5']] = data_up_o.iloc[:,[3,4,5,6,7]]
data_up[['tf1','tp1','tf2','tp2','tf3','tp3','tf4','tp4','tf5','tp5']] = data_up_tftp.iloc[:,[3,4,5,6,7,8,9,10,11,12]]
data_up[['vht1','vmt1','vht2','vmt2','vht3','vmt3','vht4','vmt4','vht5','vmt5']] = data_up_vmtvht.iloc[:,[3,4,5,6,7,8,9,10,11,12]]
# if data_up and data_do have different numbers of raws, here may be an erro
# in November, upstream lacks the record of 7/09/2017 6:05, while downstream has.So the record in downstream is deleted
data_up[['s_do1','s_do2','s_do3','s_do4','s_do5']] = data_do_fs.iloc[:,[5,7,9,11,13]]
data_up = data_up.reset_index(drop=True)

In [7]:
data_up.head()

,t,f,f1,s1,f2,s2,f3,s3,f4,s4,...,vmt3,vht4,vmt4,vht5,vmt5,s_do1,s_do2,s_do3,s_do4,s_do5
0,1/08/2017 0:00,169,61,64.3,32,71.0,50,67.4,26,62.8,...,1.1,0.0,2.2,0.0,0.0,74.8,71.0,65.7,59.5,0.0
1,1/08/2017 0:05,165,52,65.1,46,68.6,38,59.5,29,60.2,...,1.8,0.0,1.8,0.0,0.0,72.8,69.0,63.1,57.7,0.0
2,1/08/2017 0:10,147,56,65.0,36,67.5,31,58.5,24,59.5,...,1.1,0.0,1.5,0.0,0.0,72.4,67.1,61.8,56.9,0.0
3,1/08/2017 0:15,134,43,64.8,32,66.6,35,58.3,24,59.8,...,0.7,0.0,1.1,0.0,0.0,71.9,66.4,60.0,56.5,0.0
4,1/08/2017 0:20,146,50,64.9,30,67.1,39,59.2,27,58.9,...,0.7,0.0,1.5,0.0,0.0,72.0,65.4,61.4,56.8,0.0


###  1.2 Delete Outlier

In [8]:
data_up['t'] = pd.to_datetime(data_up['t'])
data = pd.DataFrame()
for month in range(11):
    
    for day in range(7): # There are 7 days data in each month
        # Select the data in the same day
        begin = data_up['t'][0] + datetime.timedelta(days=day) + relativedelta(months=month)
        end = data_up['t'][0] + datetime.timedelta(days=day+1) + relativedelta(months=month)
        interval = (data_up['t']>begin) & (data_up['t']<end)
        data_up_t = data_up[interval]

        # Computing Interquantile Range (IQR)
        upper_q_s1 = data_up_t['s1'].quantile(0.75)
        lower_q_s1 = data_up_t['s1'].quantile(0.25)
        IQR_s1 = upper_q_s1 - lower_q_s1
        upper_q_s2 = data_up_t['s2'].quantile(0.75)
        lower_q_s2 = data_up_t['s2'].quantile(0.25)
        IQR_s2 = upper_q_s2 - lower_q_s2
        upper_q_s3 = data_up_t['s3'].quantile(0.75)
        lower_q_s3 = data_up_t['s3'].quantile(0.25)
        IQR_s3 = upper_q_s3 - lower_q_s3
        upper_q_s4 = data_up_t['s4'].quantile(0.75)
        lower_q_s4 = data_up_t['s4'].quantile(0.25)
        IQR_s4 = upper_q_s4 - lower_q_s4
        upper_q_s5 = data_up_t['s5'].quantile(0.75)
        lower_q_s5 = data_up_t['s5'].quantile(0.25)
        IQR_s5 = upper_q_s5 - lower_q_s5
        
        # Delecting the records which are outlier in s1,s2,s3 and s4
        data_up_t = data_up_t[((lower_q_s1-1.5*IQR_s1) <= data_up_t['s1'])& (data_up_t['s1'] <= (upper_q_s1+1.5*IQR_s1))]
        data_up_t = data_up_t[((lower_q_s2-1.5*IQR_s2) <= data_up_t['s2'])& (data_up_t['s2'] <= (upper_q_s2+1.5*IQR_s2))]
        data_up_t = data_up_t[((lower_q_s3-1.5*IQR_s3) <= data_up_t['s3'])& (data_up_t['s3'] <= (upper_q_s3+1.5*IQR_s3))]
        data_up_t = data_up_t[((lower_q_s4-1.5*IQR_s4) <= data_up_t['s4'])& (data_up_t['s4'] <= (upper_q_s4+1.5*IQR_s4))]
        data_up_t = data_up_t[((lower_q_s5-1.5*IQR_s5) <= data_up_t['s5'])& (data_up_t['s5'] <= (upper_q_s5+1.5*IQR_s5))]

        data = data.append(data_up_t, sort = True)
        
data_up = data
data_up.describe()

,f,f1,f2,f3,f4,f5,o1,o2,o3,o4,...,vht1,vht2,vht3,vht4,vht5,vmt1,vmt2,vmt3,vmt4,vmt5
count,19974.000000,19974.000000,19974.000000,19974.000000,19974.000000,19974.000000,19974.000000,19974.000000,19974.000000,19974.000000,...,19974.0,19974.0,19974.000000,19974.000000,19974.000000,19974.0,19974.0,19974.000000,19974.000000,19974.000000
mean,267.831781,76.038300,59.985381,57.667868,41.719035,32.421198,5.417237,4.028918,4.864444,4.565285,...,0.0,0.0,0.028367,0.081982,0.008201,0.0,0.0,2.050966,3.527456,1.019646
std,199.105624,51.758306,48.815489,39.878438,34.511949,32.130384,4.047065,3.522769,3.551334,3.960542,...,0.0,0.0,0.046425,0.194124,0.027783,0.0,0.0,1.688165,2.852505,1.078159
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
25%,108.000000,37.000000,15.000000,30.000000,14.000000,0.000000,2.400000,0.900000,2.300000,1.400000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.700000,1.100000,0.000000
50%,258.000000,80.000000,53.000000,59.000000,38.000000,26.000000,5.400000,3.300000,4.900000,4.100000,...,0.0,0.0,0.000000,0.100000,0.000000,0.0,0.0,1.800000,3.400000,0.800000
75%,400.000000,109.000000,98.000000,82.000000,61.000000,50.000000,7.500000,6.500000,6.700000,6.400000,...,0.0,0.0,0.100000,0.100000,0.000000,0.0,0.0,2.900000,5.500000,1.700000
max,794.000000,207.000000,217.000000,172.000000,157.000000,146.000000,24.000000,21.200000,23.800000,20.100000,...,0.0,0.0,0.600000,2.700000,0.500000,0.0,0.0,23.900000,16.400000,19.000000


### 1.3 Construct Variables

In [9]:
# Delecting the records whose f1 = 0, f2 = 0 or f3 = 0
data_up = data_up[(data_up['f1']!=0) & (data_up['f2']!=0) & (data_up['f3']!=0) & (data_up['f4']!=0) & (data_up['f5']!=0)]

data_up.describe()
# f5 is zero during the entire Jan in 400m_5lanes. So the iteration should be corrected below at the same time.

,f,f1,f2,f3,f4,f5,o1,o2,o3,o4,...,vht1,vht2,vht3,vht4,vht5,vmt1,vmt2,vmt3,vmt4,vmt5
count,14628.000000,14628.000000,14628.000000,14628.000000,14628.000000,14628.000000,14628.000000,14628.000000,14628.000000,14628.000000,...,14628.0,14628.0,14628.000000,14628.000000,14628.000000,14628.0,14628.0,14628.000000,14628.000000,14628.000000
mean,336.533497,94.359037,74.803869,71.421452,51.710487,44.238652,6.700137,5.004820,6.007650,5.645468,...,0.0,0.0,0.035145,0.104382,0.011198,0.0,0.0,2.575875,4.422963,1.392234
std,170.272929,40.706825,43.747454,31.956511,31.316560,29.794972,3.423123,3.259304,2.961559,3.656440,...,0.0,0.0,0.048974,0.221005,0.031944,0.0,0.0,1.523460,2.564375,1.033707
min,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.100000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
25%,217.000000,68.000000,40.000000,50.000000,29.000000,22.000000,4.500000,2.500000,4.100000,3.100000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,1.500000,2.600000,0.700000
50%,308.000000,91.000000,69.000000,67.000000,46.000000,38.000000,6.100000,4.300000,5.500000,4.900000,...,0.0,0.0,0.000000,0.100000,0.000000,0.0,0.0,2.500000,4.200000,1.300000
75%,444.000000,118.000000,108.000000,90.000000,67.000000,58.000000,8.200000,7.300000,7.400000,7.100000,...,0.0,0.0,0.100000,0.100000,0.000000,0.0,0.0,3.400000,5.900000,1.800000
max,794.000000,207.000000,217.000000,172.000000,157.000000,146.000000,24.000000,21.200000,23.800000,19.900000,...,0.0,0.0,0.600000,2.700000,0.500000,0.0,0.0,23.900000,16.400000,19.000000


In [10]:
# Spliting the data into three dataframes for lane1, lane2, lane3 and lane4
lane1list = ['t']+[col for col in data_up.columns if '1' in col]
data1 = data_up[lane1list]
lane2list = ['t']+[col for col in data_up.columns if '2' in col]
data2 = data_up[lane2list]
lane3list = ['t']+[col for col in data_up.columns if '3' in col]
data3 = data_up[lane3list]
lane4list = ['t']+[col for col in data_up.columns if '4' in col]
data4 = data_up[lane4list]
lane5list = ['t']+[col for col in data_up.columns if '5' in col]
data5 = data_up[lane5list]

data1.head()

,t,f1,o1,s1,s_do1,tf1,tp1,vht1,vmt1
2017,2017-02-08 00:05:00,45,3.3,72.6,76.6,0,0,0,0
2018,2017-02-08 00:10:00,57,4.2,70.2,74.7,0,0,0,0
2019,2017-02-08 00:15:00,48,3.5,69.9,75.2,0,0,0,0
2020,2017-02-08 00:20:00,44,3.0,71.6,75.8,0,0,0,0
2021,2017-02-08 00:25:00,21,1.5,73.6,75.9,0,0,0,0


In [11]:
# Change the key of every dataframe for iteration later.
variables = ['t','f','o','s','s_do','tf','tp','vht','vmt']
data1.columns = variables
data2.columns = variables
data3.columns = variables
data4.columns = variables
data5.columns = variables
data1.loc[:,'s2'] = data2.loc[:,'s']
# Because lane2 is adjacent to lane1 and lane3 at the same time
data2.loc[:,'s1'] = data1.loc[:,'s']
data2.loc[:,'s3'] = data3.loc[:,'s']
data3.loc[:,'s2'] = data2.loc[:,'s']
data3.loc[:,'s4'] = data4.loc[:,'s']
data4.loc[:,'s3'] = data3.loc[:,'s']
data4.loc[:,'s5'] = data5.loc[:,'s']
data5.loc[:,'s4'] = data4.loc[:,'s']

data2.head()

E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,t,f,o,s,s_do,tf,tp,vht,vmt,s1,s3
2017,2017-02-08 00:05:00,20,1.3,73.8,72.0,0,0,0,0,72.6,73.4
2018,2017-02-08 00:10:00,37,2.6,71.5,71.3,0,0,0,0,70.2,68.9
2019,2017-02-08 00:15:00,27,1.8,72.1,70.9,0,0,0,0,69.9,70.0
2020,2017-02-08 00:20:00,21,1.4,72.6,71.8,0,0,0,0,71.6,70.3
2021,2017-02-08 00:25:00,11,0.7,73.1,71.7,0,0,0,0,73.6,70.3


In [12]:
# use dummy variablesS
for lanedata in [data1, data2, data3, data4, data5]:
    # ln(speed)
    lanedata.loc[:,'lns'] = lanedata['s'].map(lambda x: math.log(x))
    # truck speed
    lanedata.loc[:,'ts'] = 999
    lanedata.loc[lanedata['vht']!=0, 'ts'] =  lanedata['vmt']/lanedata['vht']
#     lanedata['ts'] = lanedata.apply(lambda x: 999.0 if x['vht'] == 0 else x['vmt']/x['vht'], axis = 1) 
#     '''axis : {0 or ‘index’, 1 or ‘columns’}, default 0
#         Axis along which the function is applied:
#         0 or ‘index’: apply function to each column.
#         1 or ‘columns’: apply function to each row.'''
    # traffic flow indicator and high truck flow indicator
    lanedata.loc[:,'fi'] = 0
    lanedata.loc[:,'htfi'] = 0
    lanedata.loc[lanedata['f']<75, 'fi'] = 1
    lanedata.loc[lanedata['tf']>100, 'htfi'] = 1
    # relative truck percentage indicator
    lanedata.loc[:,'tpia'] = 0
    lanedata.loc[:,'tpib'] = 0
    lanedata.loc[(lanedata['tp']>0.6)&(lanedata['f']<50), 'tpia'] = 1
    lanedata.loc[(lanedata['tp']<=0.2)&(lanedata['f']>200), 'tpib'] = 1
    # seasonal indicator (fall will be reference)
    lanedata.loc[:,'seasona'] = 0
    lanedata.loc[:,'seasonb'] = 0
    lanedata.loc[:,'seasonc'] = 0
    lanedata.loc[(2<lanedata['t'].dt.month) & (lanedata['t'].dt.month<6), 'seasona'] = 1 # Spring
    lanedata.loc[(5<lanedata['t'].dt.month) & (lanedata['t'].dt.month<9), 'seasonb'] = 1
    lanedata.loc[(8<lanedata['t'].dt.month) & (lanedata['t'].dt.month<12), 'seasonc'] = 1
    # time-of-week indicator
    lanedata.loc[:,'weeka'] = 0
    lanedata.loc[:,'weekb'] = 0
    lanedata.loc[:,'weekc'] = 0
    lanedata.loc[:,'weekd'] = 0
    lanedata.loc[:,'weeke'] = 0
    lanedata.loc[:,'weekf'] = 0
    lanedata.loc[lanedata['t'].dt.weekday==0, 'weeka'] = 1 # Monday
    lanedata.loc[lanedata['t'].dt.weekday==1, 'weekb'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==2, 'weekc'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==3, 'weekd'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==4, 'weeke'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==5, 'weekf'] = 1
    # time-of-day indicator
    lanedata['hour'] = lanedata['t'].dt.hour
    lanedata.loc[:,'toda'] = 0
    lanedata.loc[:,'todb'] = 0
    lanedata.loc[:,'todc'] = 0
    lanedata.loc[:,'todd'] = 0
    lanedata.loc[(0<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<6), 'toda'] = 1 # midnight to 6 am
    lanedata.loc[(7<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<9), 'todb'] = 1 # am peak
    lanedata.loc[(17<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<19), 'todc'] = 1 # pm peak
    lanedata.loc[(19<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<24), 'todd'] = 1 # night time
    
# ratio of flows in current lane to adjacent lanes
data1.loc[:,'fratio'] = data1['f']/data2['f']
# Use the average of the flow ratios with lane1 and lane3 as the ultimate flow ratio in lane2
# data2.loc[:,'fratio'] = (data2['f']/data1['f']+data2['f']/data3['f']) 
# data2.loc[:,'fratio3'] = data2['f']/data3['f']
data5.loc[:,'fratio'] = data5['f']/data4['f']

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
''' This block is used to determine the speed of lane2 is influenced by which lane (lan1 or lane3),
and the criterion is the flow ratio between lane 2 and lane1 and the flow ratio between lane 2 and lane3.
Bigger one will be dorminating!'''
# lane2
data2.loc[:,'fratio1'] = data2['f']/data1['f']
data2.loc[:,'fratio3'] = data2['f']/data3['f']
data2.loc[:,'flag'] = 0
data2.loc[data2['fratio1']<data2['fratio3'], 'flag'] = 1 # using '<' recieves better result?
data2.loc[:,'fratio'] = 0
data2.loc[data2['flag']==1,'fratio'] = data2['fratio1']
data2.loc[data2['flag']==0,'fratio'] = data2['fratio3']
# consrtuting the instruments and endogeneity dataframe for lane2
data2_instru = data1[['t','o','f','ts','fi','s_do','s']]
data2_instru.loc[data2['flag']==0,['o','f','ts','fi','s_do','s']] = data3.loc[data2['flag']==0,['o','f','ts','fi','s_do','s']]

data2_instru.loc[:,'fratio'] = 0
data2_instru.loc[:,'fratio'] = data2.loc[:,'fratio']

# lane3
data3.loc[:,'fratio2'] = data3['f']/data2['f']
data3.loc[:,'fratio4'] = data3['f']/data4['f']
data3.loc[:,'flag'] = 0
data3.loc[data3['fratio2']<data3['fratio4'], 'flag'] = 1 # using '<' recieves better result?
data3.loc[:,'fratio'] = 0
data3.loc[data3['flag']==1,'fratio'] = data3['fratio2']
data3.loc[data3['flag']==0,'fratio'] = data3['fratio4']
# consrtuting the instruments and endogeneity dataframe for lane2
data3_instru = data2[['t','o','f','ts','fi','s_do','s']]
data3_instru.loc[data3['flag']==0,['o','f','ts','fi','s_do','s']] = data4.loc[data3['flag']==0,['o','f','ts','fi','s_do','s']]

data3_instru.loc[:,'fratio'] = 0
data3_instru.loc[:,'fratio'] = data3.loc[:,'fratio']

# lane4
data4.loc[:,'fratio3'] = data4['f']/data3['f']
data4.loc[:,'fratio5'] = data4['f']/data5['f']
data4.loc[:,'flag'] = 0
data4.loc[data4['fratio3']<data4['fratio5'], 'flag'] = 1 # using '<' recieves better result?
data4.loc[:,'fratio'] = 0
data4.loc[data4['flag']==1,'fratio'] = data4['fratio3']
data4.loc[data4['flag']==0,'fratio'] = data4['fratio5']
# consrtuting the instruments and endogeneity dataframe for lane2
data4_instru = data3[['t','o','f','ts','fi','s_do','s']]
data4_instru.loc[data4['flag']==0,['o','f','ts','fi','s_do','s']] = data5.loc[data4['flag']==0,['o','f','ts','fi','s_do','s']]

data4_instru.loc[:,'fratio'] = 0
data4_instru.loc[:,'fratio'] = data4.loc[:,'fratio']

In [14]:
data4_instru.describe()

,o,f,ts,fi,s_do,s,fratio
count,14628.000000,14628.000000,14628.000000,14628.000000,14628.000000,14628.000000,14628.000000
mean,6.009865,71.468417,665.005979,0.604936,67.698236,66.936984,0.687166
std,2.965212,31.993293,456.362623,0.488881,5.899472,5.998694,0.197386
min,0.100000,1.000000,18.000000,0.000000,28.500000,27.100000,0.045455
25%,4.100000,50.000000,46.000000,0.000000,64.800000,63.200000,0.564516
50%,5.500000,67.000000,999.000000,1.000000,69.000000,68.000000,0.687500
75%,7.400000,90.000000,999.000000,1.000000,71.800000,71.500000,0.809524
max,23.800000,172.000000,999.000000,1.000000,81.800000,82.000000,5.000000


In [15]:
data1[['o','f','ts','fi','fratio','s_do','s']].describe()

,o,f,ts,fi,fratio,s_do,s
count,14628.000000,14628.000000,14628.0,14628.000000,14628.000000,14628.000000,14628.000000
mean,6.700137,94.359037,999.0,0.311457,1.563343,70.760316,70.056071
std,3.423123,40.706825,0.0,0.463105,0.860231,10.197659,7.235928
min,0.000000,1.000000,999.0,0.000000,0.333333,3.000000,33.600000
25%,4.500000,68.000000,999.0,0.000000,1.084211,67.275000,65.900000
50%,6.100000,91.000000,999.0,0.000000,1.366197,75.200000,72.400000
75%,8.200000,118.000000,999.0,1.000000,1.764706,76.400000,75.800000
max,24.000000,207.000000,999.0,1.000000,17.000000,83.400000,86.500000


In [16]:
# data1['fratio'] = data1['f']/data2['f']
# data1 = data1.dropna()
data1.dtypes

t          datetime64[ns]
f                   int64
o                 float64
s                 float64
s_do              float64
tf                  int64
tp                  int64
vht                 int64
vmt                 int64
s2                float64
lns               float64
ts                float64
fi                  int64
htfi                int64
tpia                int64
tpib                int64
seasona             int64
seasonb             int64
seasonc             int64
weeka               int64
weekb               int64
weekc               int64
weekd               int64
weeke               int64
weekf               int64
hour                int64
toda                int64
todb                int64
todc                int64
todd                int64
fratio            float64
dtype: object

## 2 Prediction 

### 2.1 Error Examination

In [17]:
# data1[['f','fratio']] = data1[['f','fratio']].infer_objects()
# data2[['f','fratio']] = data2[['f','fratio']].infer_objects()
''' Because when run the next code block, there is erro coming up: data must be numeric, string and categorical.
And the error come from other_var.
And after use data1[other_var].dtypes to visit the data type of all indexs in other_var, found the 'f' and 'fratio'
are (object). So in order to solve this problem, use dataframe.infer_objects to change the data type from object 
to int and float, respectively.
'''

" Because when run the next code block, there is erro coming up: data must be numeric, string and categorical.\nAnd the error come from other_var.\nAnd after use data1[other_var].dtypes to visit the data type of all indexs in other_var, found the 'f' and 'fratio'\nare (object). So in order to solve this problem, use dataframe.infer_objects to change the data type from object \nto int and float, respectively.\n"

In [18]:
np.linalg.matrix_rank(data1[['o','f','ts','fi','fratio']])

5

In [19]:
# def mean_absolute_percentage_error(y_true, y_pred): 
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

### 2.2 Predicting Used New Method (including downstream speed)

In [20]:
# 'exog' and 'endog' and 'instruments' cannot overlap to each other
error_result_new = pd.DataFrame(columns=['MAE1','MAE2','MAE3','MAE4','MAE5'])
speed_new1 = pd.DataFrame()
speed_new2 = pd.DataFrame()
speed_new3 = pd.DataFrame()
speed_new4 = pd.DataFrame()
speed_new5 = pd.DataFrame()
for month in range(2,11): # f5 is zero during the entire Jan in 400m_5lanes. So the iteration should be corrected at the same time. change to(2,11)
    data1_test = data1[data1['t'].dt.month == month+1]
    data1_train = data1[data1['t'].dt.month != month+1]
    data2_test = data2[data2['t'].dt.month == month+1]
    data2_train = data2[data2['t'].dt.month != month+1]
    data3_test = data3[data3['t'].dt.month == month+1]
    data3_train = data3[data3['t'].dt.month != month+1]
    data4_test = data4[data4['t'].dt.month == month+1]
    data4_train = data4[data4['t'].dt.month != month+1]
    data5_test = data5[data5['t'].dt.month == month+1]
    data5_train = data5[data5['t'].dt.month != month+1]
    
    data2_instru_test = data2_instru[data2_instru['t'].dt.month == month+1]
    data2_instru_train = data2_instru[data2_instru['t'].dt.month != month+1]
    data3_instru_test = data3_instru[data3_instru['t'].dt.month == month+1]
    data3_instru_train = data3_instru[data3_instru['t'].dt.month != month+1]
    data4_instru_test = data4_instru[data4_instru['t'].dt.month == month+1]
    data4_instru_train = data4_instru[data4_instru['t'].dt.month != month+1]
    
    other_var = ['seasona', 'seasonb', 'seasonc', 'weeka', 'weekb', 'weekc', 'weekd', 'weeke', 'weekf', 'toda', 'todb', 'todc', 'todd']
    instruments = ['o', 'f',  'ts', 'fi', 'fratio']
    data1_train.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_train.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    data3_train.rename(columns={'s':'s3', 's_do':'s_do3'}, inplace=True)
    data4_train.rename(columns={'s':'s4', 's_do':'s_do4'}, inplace=True)
    data5_train.rename(columns={'s':'s5', 's_do':'s_do5'}, inplace=True)
#     exo_var = instruments+other_var

    u1 = {'dependent': data1_train[['s1']],
         'exog': data1_train[other_var+['s_do1']],
         'endog': data1_train[['s2']],
         'instruments': data2_train[instruments+['s_do2']]}

    u2 = {'dependent': data2_train[['s2']],
         'exog': data2_train[other_var+['s_do2']],
         'endog': data2_instru_train[['s']],
         'instruments': data2_instru_train[instruments+['s_do']]}
    
    u3 = {'dependent': data3_train[['s3']],
         'exog': data3_train[other_var+['s_do3']],
         'endog': data3_instru_train[['s']],
         'instruments': data3_instru_train[instruments+['s_do']]}
    
    u4 = {'dependent': data4_train[['s4']],
         'exog': data4_train[other_var+['s_do4']],
         'endog': data4_instru_train[['s']],
         'instruments': data4_instru_train[instruments+['s_do']]}
    
    u5 = {'dependent': data5_train[['s5']],
         'exog': data5_train[other_var+['s_do5']],
         'endog': data5_train[['s4']],
         'instruments': data4_train[instruments+['s_do4']]}
    
    equations = dict(u1=u1, u2=u2, u3=u3, u4=u4, u5=u5)
    system_3sls = IV3SLS(equations)
    system_3sls_res = system_3sls.fit(method='gls', cov_type='unadjusted')
#     print(system_3sls_res)
    # system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations)
    # print(system_3sls_pre)
    
    data1_test.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_test.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    data3_test.rename(columns={'s':'s3', 's_do':'s_do3'}, inplace=True)
    data4_test.rename(columns={'s':'s4', 's_do':'s_do4'}, inplace=True)
    data5_test.rename(columns={'s':'s5', 's_do':'s_do5'}, inplace=True)
    
    u1_test = {'dependent': data1_test[['s1']],
         'exog': data1_test[other_var+['s_do1']],
         'endog': data1_test[['s2']],
         'instruments': data2_test[instruments+['s_do2']]}

    u2_test = {'dependent': data2_test[['s2']],
         'exog': data2_test[other_var+['s_do2']],
         'endog': data2_instru_test[['s']],
         'instruments': data2_instru_test[instruments+['s_do']]}
    
    u3_test = {'dependent': data3_test[['s3']],
         'exog': data3_test[other_var+['s_do3']],
         'endog': data3_instru_test[['s']],
         'instruments': data3_instru_test[instruments+['s_do']]}
    
    u4_test = {'dependent': data4_test[['s4']],
         'exog': data4_test[other_var+['s_do4']],
         'endog': data4_instru_test[['s']],
         'instruments': data4_instru_test[instruments+['s_do']]}
    
    u5_test = {'dependent': data5_test[['s5']],
         'exog': data5_test[other_var+['s_do5']],
         'endog': data5_test[['s4']],
         'instruments': data4_test[instruments+['s_do4']]}
    
    equations_test = dict(u1=u1_test, u2=u2_test, u3=u3_test, u4=u4_test, u5=u5_test)
    system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations_test)
    # system_3sls_pre.describe()
    
    # from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae
    y_true1 = data1_test['s1']
    y_pred1 = system_3sls_pre['u1']
    y_true2 = data2_test['s2']
    y_pred2 = system_3sls_pre['u2']
    y_true3 = data3_test['s3']
    y_pred3 = system_3sls_pre['u3']
    y_true4 = data4_test['s4']
    y_pred4 = system_3sls_pre['u4']
    y_true5 = data4_test['s5']
    y_pred5 = system_3sls_pre['u5']
    # print(mse(y_true, y_pred))
#     print(mae(y_true, y_pred))
#     print(mean_absolute_percentage_error(y_true, y_pred))
    error_result_new.loc[month,'MAE1'] = mae(y_true1, y_pred1)
    error_result_new.loc[month,'MAE2'] = mae(y_true2, y_pred2)
    error_result_new.loc[month,'MAE3'] = mae(y_true3, y_pred3)
    error_result_new.loc[month,'MAE4'] = mae(y_true4, y_pred4)
    error_result_new.loc[month,'MAE5'] = mae(y_true5, y_pred5)
    speed_new1 = pd.concat([speed_new1,y_pred1])
    speed_new2 = pd.concat([speed_new2,y_pred2])
    speed_new3 = pd.concat([speed_new3,y_pred3])
    speed_new4 = pd.concat([speed_new4,y_pred4])
    speed_new5 = pd.concat([speed_new5,y_pred5])
#     speed_new1['new1'] = speed_new1['new1'].append(y_pred1) # this command results in a result of a month only.

E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
E:\Anaconda3\lib\site-packages\pandas

### 2.3 Predicting Used Old Method (not including downstream speed)

In [21]:
# 'exog' and 'endog' and 'instruments' cannot overlap to each other
error_result_old = pd.DataFrame(columns=['MAE1','MAE2','MAE3','MAE4','MAE5'])
speed_old1 = pd.DataFrame()
speed_old2 = pd.DataFrame()
speed_old3 = pd.DataFrame()
speed_old4 = pd.DataFrame()
speed_old5 = pd.DataFrame()
for month in range(2,11):
    data1_test = data1[data1['t'].dt.month == month+1]
    data1_train = data1[data1['t'].dt.month != month+1]
    data2_test = data2[data2['t'].dt.month == month+1]
    data2_train = data2[data2['t'].dt.month != month+1]
    data3_test = data3[data3['t'].dt.month == month+1]
    data3_train = data3[data3['t'].dt.month != month+1]
    data4_test = data4[data4['t'].dt.month == month+1]
    data4_train = data4[data4['t'].dt.month != month+1]
    data5_test = data5[data5['t'].dt.month == month+1]
    data5_train = data5[data5['t'].dt.month != month+1]
    
    data2_instru_test = data2_instru[data2_instru['t'].dt.month == month+1]
    data2_instru_train = data2_instru[data2_instru['t'].dt.month != month+1]
    data3_instru_test = data3_instru[data3_instru['t'].dt.month == month+1]
    data3_instru_train = data3_instru[data3_instru['t'].dt.month != month+1]
    data4_instru_test = data4_instru[data4_instru['t'].dt.month == month+1]
    data4_instru_train = data4_instru[data4_instru['t'].dt.month != month+1]
    
    other_var = ['seasona', 'seasonb', 'seasonc', 'weeka', 'weekb', 'weekc', 'weekd', 'weeke', 'weekf', 'toda', 'todb', 'todc', 'todd']
    instruments = ['o', 'f',  'ts', 'fi', 'fratio']
    data1_train.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_train.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    data3_train.rename(columns={'s':'s3', 's_do':'s_do3'}, inplace=True)
    data4_train.rename(columns={'s':'s4', 's_do':'s_do4'}, inplace=True)
    data5_train.rename(columns={'s':'s5', 's_do':'s_do5'}, inplace=True)
#     exo_var = instruments+other_var

    u1 = {'dependent': data1_train[['s1']],
         'exog': data1_train[other_var],
         'endog': data1_train[['s2']],
         'instruments': data2_train[instruments]}

    u2 = {'dependent': data2_train[['s2']],
         'exog': data2_train[other_var],
         'endog': data2_instru_train[['s']],
         'instruments': data2_instru_train[instruments]}
    
    u3 = {'dependent': data3_train[['s3']],
         'exog': data3_train[other_var],
         'endog': data3_instru_train[['s']],
         'instruments': data3_instru_train[instruments]}
    
    u4 = {'dependent': data4_train[['s4']],
         'exog': data4_train[other_var],
         'endog': data4_instru_train[['s']],
         'instruments': data4_instru_train[instruments]}
    
    u5 = {'dependent': data5_train[['s5']],
         'exog': data5_train[other_var],
         'endog': data5_train[['s4']],
         'instruments': data4_train[instruments]}
    
    equations = dict(u1=u1, u2=u2, u3=u3, u4=u4, u5=u5)
    system_3sls = IV3SLS(equations)
    system_3sls_res = system_3sls.fit(method='gls', cov_type='unadjusted')
#     print(system_3sls_res)
    # system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations)
    # print(system_3sls_pre)
    
    data1_test.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_test.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    data3_test.rename(columns={'s':'s3', 's_do':'s_do3'}, inplace=True)
    data4_test.rename(columns={'s':'s4', 's_do':'s_do4'}, inplace=True)
    data5_test.rename(columns={'s':'s5', 's_do':'s_do5'}, inplace=True)
    
    u1_test = {'dependent': data1_test[['s1']],
         'exog': data1_test[other_var],
         'endog': data1_test[['s2']],
         'instruments': data2_test[instruments]}

    u2_test = {'dependent': data2_test[['s2']],
         'exog': data2_test[other_var],
         'endog': data2_instru_test[['s']],
         'instruments': data2_instru_test[instruments]}
    
    u3_test = {'dependent': data3_test[['s3']],
         'exog': data3_test[other_var],
         'endog': data3_instru_test[['s']],
         'instruments': data3_instru_test[instruments]}
    
    u4_test = {'dependent': data4_test[['s4']],
         'exog': data4_test[other_var],
         'endog': data4_instru_test[['s']],
         'instruments': data4_instru_test[instruments]}
    
    u5_test = {'dependent': data5_test[['s5']],
         'exog': data5_test[other_var],
         'endog': data5_test[['s4']],
         'instruments': data4_test[instruments]}
    
    equations_test = dict(u1=u1_test, u2=u2_test, u3=u3_test, u4=u4_test, u5=u5_test)
    system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations_test)
    # system_3sls_pre.describe()
    
    # from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae
    y_true1 = data1_test['s1']
    y_pred1 = system_3sls_pre['u1']
    y_true2 = data2_test['s2']
    y_pred2 = system_3sls_pre['u2']
    y_true3 = data3_test['s3']
    y_pred3 = system_3sls_pre['u3']
    y_true4 = data4_test['s4']
    y_pred4 = system_3sls_pre['u4']
    y_true5 = data4_test['s5']
    y_pred5 = system_3sls_pre['u5']
    # print(mse(y_true, y_pred))
#     print(mae(y_true, y_pred))
#     print(mean_absolute_percentage_error(y_true, y_pred))
    error_result_old.loc[month,'MAE1'] = mae(y_true1, y_pred1)
    error_result_old.loc[month,'MAE2'] = mae(y_true2, y_pred2)
    error_result_old.loc[month,'MAE3'] = mae(y_true3, y_pred3)
    error_result_old.loc[month,'MAE4'] = mae(y_true4, y_pred4)
    error_result_old.loc[month,'MAE5'] = mae(y_true5, y_pred5)
    speed_old1 = pd.concat([speed_old1,y_pred1])
    speed_old2 = pd.concat([speed_old2,y_pred2])
    speed_old3 = pd.concat([speed_old3,y_pred3])
    speed_old4 = pd.concat([speed_old4,y_pred4])
    speed_old5 = pd.concat([speed_old5,y_pred5])
#     speed_new1['new1'] = speed_new1['new1'].append(y_pred1) # this command results in a result of a month only.

E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
E:\Anaconda3\lib\site-packages\pandas

### 2.4 Result Exporting

In [22]:
# speed_new1.columns = ['new1']
# speed_new2.columns = ['new2']
# speed_new3.columns = ['new3']
# speed_new4.columns = ['new4']
# speed_new5.columns = ['new5']
# speed_old1.columns = ['old1']
# speed_old2.columns = ['old2']
# speed_old3.columns = ['old3']
# speed_old4.columns = ['old4']
# speed_old5.columns = ['old5']

In [23]:
speed = speed_new1
speed['new2'] = speed_new2
speed['new3'] = speed_new3
speed['new4'] = speed_new4
speed['new5'] = speed_new5
speed['old1'] = speed_old1
speed['old2'] = speed_old2
speed['old3'] = speed_old3
speed['old4'] = speed_old4
speed['old5'] = speed_old5
speed['observed1'] = data1['s']
speed['observed2'] = data2['s']
speed['observed3'] = data3['s']
speed['observed4'] = data4['s']
speed['observed5'] = data5['s']
speed.rename(columns={0: 'new1'}, inplace=True)
speed.describe()

,new1,new2,new3,new4,new5,old1,old2,old3,old4,old5,observed1,observed2,observed3,observed4,observed5
count,13179.000000,13179.000000,13179.000000,13179.000000,13179.000000,13179.000000,13179.000000,13179.000000,13179.000000,13179.000000,13179.000000,13179.000000,13179.000000,13179.000000,13179.000000
mean,69.944023,68.113897,66.820199,63.958445,62.026237,70.068709,68.125093,66.874520,63.969004,62.076840,70.025624,68.133978,66.854579,63.931156,62.106085
std,10.492237,6.948750,5.882721,4.771847,8.741573,11.805549,6.929113,12.285616,6.299545,15.183289,7.280632,9.996534,5.960998,11.193308,4.255468
min,1.876805,32.067998,34.287421,33.157030,13.153212,-1.766622,31.916997,-9.115183,24.332690,-25.367750,33.600000,3.000000,27.100000,3.000000,35.900000
25%,65.507041,64.484037,63.864029,61.287460,59.807269,64.875019,64.473344,61.746197,60.872565,57.389434,66.000000,64.200000,63.200000,62.700000,61.300000
50%,73.969107,70.323850,68.221493,64.445830,62.981333,73.866748,69.736124,69.529739,64.478935,62.172078,72.400000,72.800000,67.900000,66.800000,63.800000
75%,75.957462,73.130792,70.956920,67.304512,66.383483,76.269433,73.333666,73.598281,68.267012,68.270898,75.800000,73.900000,71.400000,69.000000,64.600000
max,84.181765,83.068373,78.861449,84.547381,76.954696,92.129606,81.426315,96.895010,80.810699,98.085199,86.500000,80.900000,82.000000,79.200000,70.600000


In [24]:
error_result = pd.DataFrame()
error_result['MAE_new1'] = error_result_new['MAE1']
error_result['MAE_new2'] = error_result_new['MAE2']
error_result['MAE_new3'] = error_result_new['MAE3']
error_result['MAE_new4'] = error_result_new['MAE4']
error_result['MAE_new5'] = error_result_new['MAE5']
error_result['MAE_old1'] = error_result_old['MAE1']
error_result['MAE_old2'] = error_result_old['MAE2']
error_result['MAE_old3'] = error_result_old['MAE3']
error_result['MAE_old4'] = error_result_old['MAE4']
error_result['MAE_old5'] = error_result_old['MAE5']
error_result

,MAE_new1,MAE_new2,MAE_new3,MAE_new4,MAE_new5,MAE_old1,MAE_old2,MAE_old3,MAE_old4,MAE_old5
2,4.02152,3.74454,2.94464,4.35776,4.13775,4.85048,4.14591,6.52885,4.4824,7.62885
3,2.06076,2.17609,2.37994,5.58037,4.17197,3.17132,2.79457,6.68906,4.95938,8.41821
4,6.24915,5.93795,2.61996,6.3831,6.41987,7.04118,6.19599,8.93857,6.45621,10.3152
5,4.01499,3.86043,2.7788,6.13882,5.22037,5.41254,4.62384,7.14522,7.59419,10.8323
6,2.95959,2.7767,2.77125,4.09801,3.47907,3.9946,3.52074,6.54779,4.69726,7.35172
7,2.55587,2.53163,2.02621,4.09629,3.2044,4.19607,3.60123,6.17076,3.69152,6.85984
8,5.8241,5.74876,3.13209,5.45868,4.60657,6.61524,5.90128,10.0322,6.41775,8.96864
9,3.77425,3.86569,2.61433,5.44207,6.04496,5.52713,4.63044,7.88848,5.85777,9.12969
10,3.33994,3.09168,3.07373,4.00166,3.28183,4.18025,3.64196,7.71532,5.10268,6.7466


In [25]:
speed.to_excel('G:\\Modeling Endogeneity\\Results\\400m_5lanes_speed.xlsx')
error_result.to_excel('G:\\Modeling Endogeneity\\Results\\400m_5lanes_error.xlsx')
data_up[['t','f1','f2','f3','f4','f5']].to_excel('G:\\Modeling Endogeneity\\Results\\400m_5lanes_flow.xlsx')